# Estudo Algoritmo de Classificação Naive Bayes
- O objetivo deste notebook é o estudo do algoritmo de classificação Naive Bayes do scikit-learn.
- Vamos utilizar nesse estudo o data set 'credit-data.csv'.
- Este dataset contém 5 atributos e 2.000 instâncias (linhas). 

  <h2>Principais Vantagens x Desvantagens </h2>
- <h3>Vantagens </h3>
- Rápido
- Simplicidade de interpretação
- Trabalha com altas dimensões
- Boas previsões em bases pequenas
-  <h3>Desvantagem </h3>
- Combinação de características (atributos independentes) – cada par de características são independentes (nem sempre é verdade)

In [1]:
# Importando as bibliotecas iniciais
import pandas as pd
import numpy as np

In [2]:
# Importa o dataset
base = pd.read_csv('./credit-data.csv')


In [3]:
# Verificando o shape do dataset
base.shape


(2000, 5)

In [4]:
# Visualiza as primeira linhas
base.head()

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


# Pré-processamento - Tratamento dos dados

In [5]:
# Trata as idades negativas 
base.loc[base.age<0, 'age'] = 40.92

In [6]:
# Seleciona os atributos previsores
previsores = base.iloc[:, 1:4].values
previsores

array([[  6.61559251e+04,   5.90170151e+01,   8.10653213e+03],
       [  3.44151540e+04,   4.81171531e+01,   6.56474502e+03],
       [  5.73171701e+04,   6.31080495e+01,   8.02095330e+03],
       ..., 
       [  4.43114493e+04,   2.80171669e+01,   5.52278669e+03],
       [  4.37560566e+04,   6.39717958e+01,   1.62272260e+03],
       [  6.94365796e+04,   5.61526170e+01,   7.37883360e+03]])

In [7]:
# Seleciona a classe alvo (target)
classe = base.iloc[:,4]

In [8]:
# SimpleImputer para tratar os valores não informados - desconhecidos
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(previsores[:,1:4])
previsores[:,1:4] = imp_mean.transform(previsores[:,1:4] )


In [9]:
# StandardScaler para coloca todos os valores na mesma escala
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [10]:
# Separa os dados pata treinamento e teste
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe , test_size = 0.25, random_state = 0)


# Criando o Modelo de Previsão

In [11]:
# Cria o modelo de previsão Naive Bayes
from sklearn.naive_bayes import GaussianNB
classificador = GaussianNB()
classificador.fit(previsores_treinamento, classe_treinamento)

GaussianNB(priors=None, var_smoothing=1e-09)

In [12]:
# Realizando o testes do modelo
classe_prevista_teste = classificador.predict(previsores_teste)

# Verificando a performance do Modelo

In [13]:
# Analisando a performance do Modelo
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
precisao_modelo = accuracy_score(classe_teste, classe_prevista_teste)
print('Acurácia do Modelo: %.2f%% ' % (precisao_modelo*100))


Acurácia do Modelo: 93.80% 


In [14]:
# Relatório de Classificação
from sklearn.metrics import classification_report

#precision -  daqueles que classifiquei como corretos, quantos efetivamente estavam corretos? 
#recall    -  quando realmente é da classe XPTO, o quão frequente você classifica como XPTO?
#f1-score  -  balanço entre a precisão e o recall. Combina precisão e recall em um número único que indique a qualidade geral do seu modelo (quanto maior melhor o modelo).
#support   -  número de ocorrência de cada classe.

relatorio_classificacao = classification_report(classe_teste, classe_prevista_teste ,digits=4)

# Imprimindo o relatório
print('\tRelatório de Classificação do modelo')
print(relatorio_classificacao)

	Relatório de Classificação do modelo
              precision    recall  f1-score   support

           0     0.9490    0.9817    0.9651       436
           1     0.8367    0.6406    0.7257        64

   micro avg     0.9380    0.9380    0.9380       500
   macro avg     0.8929    0.8111    0.8454       500
weighted avg     0.9346    0.9380    0.9344       500



In [15]:
# Visualizando a Matriz de confusão (confusion matrix)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(classe_teste, classe_prevista_teste) 

df_cm = pd.DataFrame(cm)

print('\n')
print('Obs.: Nas linhas temos as Classes reais e nas colunas as Classes previstas.\n')
print('Matriz de Confusão')
df_cm



Obs.: Nas linhas temos as Classes reais e nas colunas as Classes previstas.

Matriz de Confusão


,0,1
0,428,8
1,23,41


In [16]:
# Comparando a Classe Prevista com a Classe Real

df = pd.DataFrame( )
df['Classe_Prevista'] = classe_prevista_teste
df['Classe_Real'] = classe_teste.values
df['Comparacao'] = df['Classe_Prevista'] == df['Classe_Real']

df_comparacao = df.groupby(by='Comparacao').count()

print('O modelo ACERTOU %d classes e ERROU %d classes.\n' % (df_comparacao.iloc[[1], [0]].values, df_comparacao.iloc[[0], [0]].values))

df_comparacao

O modelo ACERTOU 469 classes e ERROU 31 classes.



,Classe_Prevista,Classe_Real
Comparacao,,
False,31,31
True,469,469
